# Evolver Loop 11 Analysis

## Key Findings:
1. exp_010 (safe ensemble) has CV=70.365 but NOT YET SUBMITTED
2. 4/6 submissions FAILED Kaggle validation (67% failure rate)
3. External SmartManoj data downloaded - need to compare per-N scores
4. Top kernel uses C++ optimizer with fractional translation

## Priority: Submit exp_010 to validate the "safe" approach

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from numba import njit
import math

# Load baseline and exp_010
baseline_path = '/home/code/experiments/001_valid_baseline/submission.csv'
exp010_path = '/home/code/experiments/010_safe_ensemble/submission.csv'
external_path = '/home/code/external_smartmanoj.csv'

print('Loading submissions...')
baseline = pd.read_csv(baseline_path)
exp010 = pd.read_csv(exp010_path)
external = pd.read_csv(external_path)

print(f'Baseline rows: {len(baseline)}')
print(f'Exp010 rows: {len(exp010)}')
print(f'External rows: {len(external)}')

# Check for 's' prefix
print(f"\nBaseline x sample: {baseline['x'].iloc[0]}")
print(f"Exp010 x sample: {exp010['x'].iloc[0]}")
print(f"External x sample: {external['x'].iloc[0]}")

In [ ]:
# Score calculation
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs,ys,degs,tx,ty):
    n=xs.size; V=tx.size
    mnx=1e300; mny=1e300; mxx=-1e300; mxy=-1e300
    for i in range(n):
        r=degs[i]*math.pi/180.0
        c=math.cos(r); s=math.sin(r)
        xi=xs[i]; yi=ys[i]
        for j in range(V):
            X=c*tx[j]-s*ty[j]+xi
            Y=s*tx[j]+c*ty[j]+yi
            if X<mnx: mnx=X
            if X>mxx: mxx=X
            if Y<mny: mny=Y
            if Y>mxy: mxy=Y
    side=max(mxx-mnx,mxy-mny)
    return side*side/ n

def strip(a):
    return np.array([float(str(v).replace('s','')) for v in a],np.float64)

tx, ty = make_polygon_template()

def compute_per_n_scores(df):
    df = df.copy()
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    scores = {}
    for n, g in df.groupby('N'):
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        scores[n] = score_group(xs, ys, ds, tx, ty)
    return scores

print('Computing per-N scores...')
baseline_scores = compute_per_n_scores(baseline)
exp010_scores = compute_per_n_scores(exp010)
external_scores = compute_per_n_scores(external)

print(f'Baseline total: {sum(baseline_scores.values()):.6f}')
print(f'Exp010 total: {sum(exp010_scores.values()):.6f}')
print(f'External total: {sum(external_scores.values()):.6f}')

In [ ]:
# Compare per-N: Where is external BETTER than exp010?
print('\n=== External vs Exp010 (where external is better) ===')
external_better = []
for n in range(1, 201):
    diff = exp010_scores[n] - external_scores[n]
    if diff > 1e-6:  # External is better
        external_better.append((n, diff, exp010_scores[n], external_scores[n]))
        if diff > 0.001:
            print(f'N={n}: exp010={exp010_scores[n]:.6f}, external={external_scores[n]:.6f}, diff={diff:.6f}')

print(f'\nTotal N values where external is better: {len(external_better)}')
print(f'Total potential improvement: {sum(d[1] for d in external_better):.6f}')

# Where is exp010 BETTER than external?
exp010_better = []
for n in range(1, 201):
    diff = external_scores[n] - exp010_scores[n]
    if diff > 1e-6:  # Exp010 is better
        exp010_better.append((n, diff))

print(f'\nTotal N values where exp010 is better: {len(exp010_better)}')
print(f'Total exp010 advantage: {sum(d[1] for d in exp010_better):.6f}')

In [ ]:
# Create best-of-both ensemble
print('\n=== Creating best-of-both ensemble ===')

best_scores = {}
best_source = {}
for n in range(1, 201):
    if exp010_scores[n] <= external_scores[n]:
        best_scores[n] = exp010_scores[n]
        best_source[n] = 'exp010'
    else:
        best_scores[n] = external_scores[n]
        best_source[n] = 'external'

print(f'Best ensemble total: {sum(best_scores.values()):.6f}')
print(f'From exp010: {sum(1 for s in best_source.values() if s == "exp010")}')
print(f'From external: {sum(1 for s in best_source.values() if s == "external")}')

# Improvement over exp010
improvement = sum(exp010_scores.values()) - sum(best_scores.values())
print(f'\nImprovement over exp010: {improvement:.6f}')

In [ ]:
# Check what N values have the biggest improvements from external
print('\n=== Top 20 improvements from external ===')
external_better_sorted = sorted(external_better, key=lambda x: x[1], reverse=True)[:20]
for n, diff, exp_score, ext_score in external_better_sorted:
    print(f'N={n}: improvement={diff:.6f} (exp010={exp_score:.6f}, external={ext_score:.6f})')